# Notebook for CME Futures Challenge

### The Rough Idea

Model indices as geometric brownian motion (dS/S = mudt + sigmadB)  
Model mu (market line) as a linear regression with numerous factors including economic, credit measures, etc  
Model sigma as a function of volatility including recent volatility and EMA (decay)  
Long/short based on futures mispricings based on our model  

# Downloading historical data for indices (S&P, NASDAQ, DJIA)

Imports

In [407]:
import yfinance as yf
import pandas as pd
import plotly.express as px
from typing import List, Dict

Make get_data function for downloading from yf

In [408]:
timeframe = '1000mo' # set timeframe

def get_data(tickers: List):
    data_dictionary = {}
    for ticker in tickers:
        data_dictionary[ticker] = yf.download(ticker, period=timeframe, interval='1d')
    return data_dictionary

Now let's get data for indices and display with pd

In [409]:
indices = ['^GSPC', '^IXIC', '^DJI'] # S&P, NASDAQ, DJIA
etfs = ['SPY', 'QQQ', 'DIA']
data_dictionary = get_data(etfs)

#s_p = pd.DataFrame(data_dictionary['^GSPC'])
#nasdaq = pd.DataFrame(data_dictionary['^IXIC'])
#djia = pd.DataFrame(data_dictionary['^DJI'])

s_p = pd.DataFrame(data_dictionary['SPY'])
nasdaq = pd.DataFrame(data_dictionary['QQQ'])
djia = pd.DataFrame(data_dictionary['DIA'])

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_18108\3981970906.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_18108\3981970906.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_18108\3981970906.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [410]:
s_p

Price,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY
Date,,,,,
1993-01-29,24.313030,24.330323,24.209276,24.330323,1003200
1993-02-01,24.485970,24.485970,24.330338,24.330338,480500
1993-02-02,24.537834,24.555127,24.416788,24.468665,201300
1993-02-03,24.797218,24.814511,24.555126,24.572418,529400
1993-02-04,24.900988,24.952865,24.607018,24.883695,531500
...,...,...,...,...,...
2025-09-15,659.082703,659.212348,657.517097,657.816255,63772400
2025-09-16,658.175232,659.950340,657.387438,659.641138,61169000


We need to flatten this - notice ticker header

In [411]:
s_p = s_p.droplevel(1, axis=1)
nasdaq = nasdaq.droplevel(1, axis=1)
djia = djia.droplevel(1, axis=1)

In [412]:
s_p

Price,Close,High,Low,Open,Volume
Date,,,,,
1993-01-29,24.313030,24.330323,24.209276,24.330323,1003200
1993-02-01,24.485970,24.485970,24.330338,24.330338,480500
1993-02-02,24.537834,24.555127,24.416788,24.468665,201300
1993-02-03,24.797218,24.814511,24.555126,24.572418,529400
1993-02-04,24.900988,24.952865,24.607018,24.883695,531500
...,...,...,...,...,...
2025-09-15,659.082703,659.212348,657.517097,657.816255,63772400
2025-09-16,658.175232,659.950340,657.387438,659.641138,61169000
2025-09-17,657.357544,659.890500,652.491031,658.185266,101952200


Let's drop high, low, and open and rename columns

In [413]:
s_p.drop(columns=['High', 'Low', 'Open'], inplace=True)
nasdaq.drop(columns=['High', 'Low', 'Open'], inplace=True)
djia.drop(columns=['High', 'Low', 'Open'], inplace=True)

s_p = s_p.rename(columns={'Close': 'S&P_Close', 'Volume': 'S&P_Volume'})
nasdaq = nasdaq.rename(columns={'Close': 'NASDAQ_Close', 'Volume': 'NASDAQ_Volume'})
djia = djia.rename(columns={'Close': 'DJIA_Close', 'Volume': 'DJIA_Volume'})

Let's get a quick plot of an index

In [414]:
fig = px.line(s_p, x=s_p.index, y="S&P_Close", title="S&P Daily Past 30 Years")
fig.show()

# Downloading historical data for our factor model

We are going to model the index as a geometric brownian motion, with the mu factor being a linear regression model with numerous inputs.  

## Factor considerations:  
### <u>Term structure</u>
###### Term spread (10Y-3M)

### <u>Credit conditions</u>
###### IG spread (BAA-AAA)

### <u>Valuation</u>
###### Forward E/P - real 10Y
###### Dividend yield

### <u>Economic</u>
###### Fed funds
###### Inflation (CPI)
###### DXY change (dollar index)  

### Some of these we can get from yahoo finance:  

In [415]:
tickers = [
    # Term structure
    '^TNX', # 10yr CBOE
    '^IRX', # 3m bill (on discount basis, need to convert to yield)

    # Economic
    'DX-Y.NYB', # Dollar index
]

data_dictionary = get_data(tickers)

ten_yr = pd.DataFrame(data_dictionary['^TNX']['Close'])
three_m = pd.DataFrame(data_dictionary['^IRX']['Close'])
dollar_index = pd.DataFrame(data_dictionary['DX-Y.NYB']['Close'])

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_18108\3981970906.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_18108\3981970906.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_18108\3981970906.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


Rename columns

In [416]:
ten_yr = ten_yr.rename(columns={'^TNX': 'ten_yr'})
three_m = three_m.rename(columns={'^IRX': 'three_m'})
dollar_index = dollar_index.rename(columns={'DX-Y.NYB': 'dollar_index'})

We should get dividend yield too

In [417]:
etfs = ['SPY', 'QQQ', 'DIA']
div_data = {}

for etf in etfs:
    ticker = yf.Ticker(etf)
    div = ticker.dividends
    price = ticker.history(timeframe)['Close']

    # Calculate dividend yield
    div_12m = div.rolling(window='365D', min_periods=1).sum()
    div_12m = div_12m.reindex(price.index, method='ffill')
    div_yield = div_12m / price
    div_data[etf] = div_yield

Fix index for all 3 and rename columns

In [418]:
div_data['SPY'].index = pd.to_datetime(div_data['SPY'].index).normalize().tz_localize(None) # Normalize puts date in format we want
div_data['QQQ'].index = pd.to_datetime(div_data['QQQ'].index).normalize().tz_localize(None) # Localize (none) makes sure it doesn't add our timezone
div_data['DIA'].index = pd.to_datetime(div_data['DIA'].index).normalize().tz_localize(None)

div_data['SPY'].name = 'SPY_div'
div_data['QQQ'].name = 'QQQ_div'
div_data['DIA'].name = 'DIA_div'

In [419]:
div_data['SPY']

Date
1993-01-29         NaN
1993-02-01         NaN
1993-02-02         NaN
1993-02-03         NaN
1993-02-04         NaN
                ...   
2025-09-15    0.013546
2025-09-16    0.013565
2025-09-17    0.013582
2025-09-18    0.013518
2025-09-19    0.013560
Name: SPY_div, Length: 8217, dtype: float64

### pandas_datareader lets us download fred data

In [420]:
from pandas_datareader import data as pdr
from datetime import datetime

In [421]:
start = datetime(1990,1,1) # Start date for download

# Macroeconomic data
gdp = pdr.DataReader("GDP", "fred", start)
cpi = pdr.DataReader("CPIAUCSL", "fred", start)
fedfunds = pdr.DataReader("FEDFUNDS", "fred", start)

# For some reason this download doesn't have the most recent fed funds rate
fedfunds = pd.concat([fedfunds['FEDFUNDS'], pd.Series([4.08], index=[datetime(2025,9,17)])])

# Credit risk data
ig_spread = pdr.DataReader("BAMLC0A4CBBB", "fred", start)   # BofA BBB corp minus Treasuries
#hy_spread = pdr.DataReader("BAMLH0A0HYM2", "fred", start)   # BofA US High Yield spread
#baa_spread = pdr.DataReader("BAA10Y", "fred", start)        # Moody’s Baa – 10Y Treasury

Rename series

In [422]:
cpi.name = 'CPI'
fedfunds.name = 'fed_funds'
ig_spread.name = 'credit_spread'

In [423]:
fred_data = [gdp, cpi, fedfunds, ig_spread]

# Last business day <= today
last_bday = pd.bdate_range(end=pd.Timestamp.today().normalize().tz_localize(None), periods=1)[0]

for i, df in enumerate(fred_data):
    s = df.squeeze() # make it a Series
    # Build a business-day index from the series start to last_bday
    bidx = pd.bdate_range(start=s.index.min(), end=last_bday)
    # Reindex to business days and forward-fill
    s = s.reindex(bidx, method='ffill')
    # Write back as a 1-col DataFrame with a proper name
    name = s.name if s.name else f"series_{i}"
    fred_data[i] = s.to_frame(name)

In [424]:
fred_data[0]

,GDP
1990-01-01,5872.701
1990-01-02,5872.701
1990-01-03,5872.701
1990-01-04,5872.701
1990-01-05,5872.701
...,...
2025-09-15,30353.902
2025-09-16,30353.902
2025-09-17,30353.902
2025-09-18,30353.902


Let's build a master dataframe

In [425]:
data = s_p.join([nasdaq, djia, div_data['SPY'], div_data['QQQ'], div_data['DIA'], ten_yr, three_m, dollar_index, fred_data[0], fred_data[1], fred_data[2], fred_data[3]])
data

,S&P_Close,S&P_Volume,NASDAQ_Close,NASDAQ_Volume,DJIA_Close,DJIA_Volume,SPY_div,QQQ_div,DIA_div,ten_yr,three_m,dollar_index,GDP,CPIAUCSL,fed_funds,BAMLC0A4CBBB
Date,,,,,,,,,,,,,,,,
1993-01-29,24.313030,1003200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.390,2.900,92.459999,6729.459,142.800,3.02,NaN
1993-02-01,24.485970,480500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.380,2.900,93.559998,6729.459,143.100,3.03,NaN
1993-02-02,24.537834,201300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.460,2.960,93.919998,6729.459,143.100,3.03,NaN
1993-02-03,24.797218,529400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.450,2.930,94.239998,6729.459,143.100,3.03,NaN
1993-02-04,24.900988,531500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.390,2.900,94.529999,6729.459,143.100,3.03,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-15,659.082703,63772400.0,591.679993,44360300.0,458.759308,4862200.0,0.013546,0.006052,0.015259,4.034,3.900,97.300003,30353.902,323.364,4.33,0.95
2025-09-16,658.175232,61169000.0,591.179993,36942100.0,457.472961,4526100.0,0.013565,0.006057,0.015301,4.026,3.890,96.629997,30353.902,323.364,4.33,0.96
2025-09-17,657.357544,101952200.0,590.000000,69384800.0,459.945923,5998000.0,0.013582,0.006069,0.015219,4.076,3.868,96.870003,30353.902,323.364,4.08,0.96


# Linear regression model

### Feature Engineering

We need to be careful to not include things such as raw moving averages that will leak volatility information into our drift prediction  

In [426]:
import numpy as np

Function definitions to help out

In [427]:
def rolling_mean(data, window):
    return data.rolling(window, min_periods=window).mean()

Features

#  (TODO: look at making features like diffs for economic metrics, figure out when economic metrics are released vs reported in data)

In [ ]:
# First, make log prices / volumes of our data, then log normal assumptions are better and everything is additive
data['S&P_log_price'] = np.log(data['S&P_Close'])
data['NASDAQ_log_price'] = np.log(data['NASDAQ_Close'])
data['DJIA_log_price'] = np.log(data['DJIA_Close'])

data['S&P_log_volume'] = np.log(data['S&P_Volume'])
data['NASDAQ_log_volume'] = np.log(data['NASDAQ_Volume'])
data['DJIA_log_volume'] = np.log(data['DJIA_Volume'])

Setting our target returns metric

In [429]:
# Log returns (21 = 1 month)
days = 21
data['S&P_ret'] = data['S&P_log_price'].diff(days)
data['NASDAQ_ret'] = data['NASDAQ_log_price'].diff(days)
data['DJIA_ret'] = data['DJIA_log_price'].diff(days)

# Next month log returns -- This will be our target variable
data[['S&P_next_ret','NASDAQ_next_ret','DJIA_next_ret']] = data[['S&P_ret','NASDAQ_ret','DJIA_ret']].shift(-days).dropna()

ETF Features

In [430]:
# ===== S&P =====
# Price-based
data['S&P_mom_1w'] = data['S&P_log_price'].diff(5) # Total price change / momentum indicator
data['S&P_mom_3m'] = data['S&P_log_price'].diff(63)
data['S&P_3m_rolling_price'] = rolling_mean(data['S&P_log_price'], 63)
data['S&P_trend_speed_price'] = data['S&P_3m_rolling_price'].diff(5)  # How fast the 3m trend is changing on a weekly basis
data['S&P_trend_dist_price'] = data['S&P_log_price'] - data['S&P_3m_rolling_price']


# Volume-based (essentially the same as price for now)
data['S&P_vlm_1w'] = data['S&P_log_volume'].diff(5) # Total volume change / momentum indicator
data['S&P_vlm_1m'] = data['S&P_log_volume'].diff(21)
data['S&P_vlm_3m'] = data['S&P_log_volume'].diff(63)
data['S&P_3m_rolling_volume'] = rolling_mean(data['S&P_log_volume'], 63)
data['S&P_trend_speed_volume'] = data['S&P_3m_rolling_volume'].diff(5)  # How fast the 3m trend is changing on a weekly basis
data['S&P_trend_dist_volume'] = data['S&P_log_volume'] - data['S&P_3m_rolling_volume']

# ===== NASDAQ =====
# Price-based
data['NASDAQ_mom_1w'] = data['NASDAQ_log_price'].diff(5)  # Total price change / momentum indicator
data['NASDAQ_mom_3m'] = data['NASDAQ_log_price'].diff(63)
data['NASDAQ_3m_rolling_price'] = rolling_mean(data['NASDAQ_log_price'], 63)
data['NASDAQ_trend_speed_price'] = data['NASDAQ_3m_rolling_price'].diff(5)  # How fast the 3m trend is changing on a weekly basis
data['NASDAQ_trend_dist_price'] = data['NASDAQ_log_price'] - data['NASDAQ_3m_rolling_price']

# Volume-based (essentially the same as price for now)
data['NASDAQ_vlm_1w'] = data['NASDAQ_log_volume'].diff(5)  # Total volume change / momentum indicator
data['NASDAQ_vlm_1m'] = data['NASDAQ_log_volume'].diff(21)
data['NASDAQ_vlm_3m'] = data['NASDAQ_log_volume'].diff(63)
data['NASDAQ_3m_rolling_volume'] = rolling_mean(data['NASDAQ_log_volume'], 63)
data['NASDAQ_trend_speed_volume'] = data['NASDAQ_3m_rolling_volume'].diff(5)  # How fast the 3m trend is changing on a weekly basis
data['NASDAQ_trend_dist_volume'] = data['NASDAQ_log_volume'] - data['NASDAQ_3m_rolling_volume']

# ===== DJIA =====
# Price-based
data['DJIA_mom_1w'] = data['DJIA_log_price'].diff(5)  # Total price change / momentum indicator
data['DJIA_mom_3m'] = data['DJIA_log_price'].diff(63)
data['DJIA_3m_rolling_price'] = rolling_mean(data['DJIA_log_price'], 63)
data['DJIA_trend_speed_price'] = data['DJIA_3m_rolling_price'].diff(5)  # How fast the 3m trend is changing on a weekly basis
data['DJIA_trend_dist_price'] = data['DJIA_log_price'] - data['DJIA_3m_rolling_price']

# Volume-based (essentially the same as price for now)
data['DJIA_vlm_1w'] = data['DJIA_log_volume'].diff(5)  # Total volume change / momentum indicator
data['DJIA_vlm_1m'] = data['DJIA_log_volume'].diff(21)
data['DJIA_vlm_3m'] = data['DJIA_log_volume'].diff(63)
data['DJIA_3m_rolling_volume'] = rolling_mean(data['DJIA_log_volume'], 63)
data['DJIA_trend_speed_volume'] = data['DJIA_3m_rolling_volume'].diff(5)  # How fast the 3m trend is changing on a weekly basis
data['DJIA_trend_dist_volume'] = data['DJIA_log_volume'] - data['DJIA_3m_rolling_volume']

Macro features

Other features

In [431]:
month_dummies = pd.get_dummies(data.index.month, prefix="month")
month_dummies.set_index(data.index, inplace=True)
data = data.join(month_dummies)

In [432]:
data

,S&P_Close,S&P_Volume,NASDAQ_Close,NASDAQ_Volume,DJIA_Close,DJIA_Volume,SPY_div,QQQ_div,DIA_div,ten_yr,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
Date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,24.313030,1003200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.390,...,False,False,False,False,False,False,False,False,False,False
1993-02-01,24.485970,480500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.380,...,False,False,False,False,False,False,False,False,False,False
1993-02-02,24.537834,201300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.460,...,False,False,False,False,False,False,False,False,False,False
1993-02-03,24.797218,529400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.450,...,False,False,False,False,False,False,False,False,False,False
1993-02-04,24.900988,531500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.390,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-15,659.082703,63772400.0,591.679993,44360300.0,458.759308,4862200.0,0.013546,0.006052,0.015259,4.034,...,False,False,False,False,False,False,True,False,False,False
2025-09-16,658.175232,61169000.0,591.179993,36942100.0,457.472961,4526100.0,0.013565,0.006057,0.015301,4.026,...,False,False,False,False,False,False,True,False,False,False
2025-09-17,657.357544,101952200.0,590.000000,69384800.0,459.945923,5998000.0,0.013582,0.006069,0.015219,4.076,...,False,False,False,False,False,False,True,False,False,False


### Preprocessing Data

Let's check for NaNs

In [433]:
data.isna().sum()

S&P_Close           0
S&P_Volume          0
NASDAQ_Close     1542
NASDAQ_Volume    1542
DJIA_Close       1256
                 ... 
month_8             0
month_9             0
month_10            0
month_11            0
month_12            0
Length: 73, dtype: int64

Impute some NaNs with average

In [ ]:
data['ten_yr'] = data['ten_yr'].fillna(data['ten_yr'].mean())
data['three_m'] = data['three_m'].fillna(data['three_m'].mean())
data['dollar_index'] = data['dollar_index'].fillna(data['dollar_index'].mean())
data['BAMLC0A4CBBB'] = data['BAMLC0A4CBBB'].fillna(data['BAMLC0A4CBBB'].mean())
#data['S&P_ret'] = data['S&P_ret'].fillna(data['S&P_ret'].mean())
#data['NASDAQ_ret'] = data['NASDAQ_ret'].fillna(data['NASDAQ_ret'].mean())
#data['DJIA_ret'] = data['DJIA_ret'].fillna(data['DJIA_ret'].mean())

Drop others

In [435]:
data = data.dropna()
data

,S&P_Close,S&P_Volume,NASDAQ_Close,NASDAQ_Volume,DJIA_Close,DJIA_Volume,SPY_div,QQQ_div,DIA_div,ten_yr,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
Date,,,,,,,,,,,,,,,,,,,,,
2003-12-24,73.179512,8055800.0,30.408176,44840300.0,63.607685,3742600.0,0.028232,0.000460,0.033565,4.187,...,False,False,False,False,False,False,False,False,False,True
2003-12-26,73.232903,8308400.0,30.365812,25497600.0,63.650841,2088500.0,0.028211,0.000461,0.033542,4.148,...,False,False,False,False,False,False,False,False,False,True
2003-12-29,74.207542,22483700.0,30.865555,63296200.0,64.519875,6589900.0,0.027841,0.000454,0.033091,4.230,...,False,False,False,False,False,False,False,False,False,True
2003-12-30,74.220940,19559500.0,30.967211,48249400.0,64.304161,5478200.0,0.027836,0.000452,0.033202,4.279,...,False,False,False,False,False,False,False,False,False,True
2003-12-31,74.287674,31501800.0,30.882515,60494600.0,64.452087,5658100.0,0.027811,0.000453,0.033125,4.257,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-14,643.166870,59327500.0,579.890015,45425000.0,447.615112,4553200.0,0.013881,0.006175,0.015478,4.293,...,False,False,False,False,False,True,False,False,False,False
2025-08-15,641.661072,68592500.0,577.340027,49480200.0,448.249329,5584400.0,0.013914,0.006203,0.015616,4.328,...,False,False,False,False,False,True,False,False,False,False
2025-08-18,641.521423,43804900.0,577.109985,29831000.0,447.770691,3809100.0,0.013917,0.006205,0.015633,4.341,...,False,False,False,False,False,True,False,False,False,False


### Split data

Training/testing 80/20 split

In [436]:
import math

In [ ]:
cutoff = math.floor(len(data)*.8)
training_data = data.iloc[:cutoff]
testing_data = data.iloc[cutoff:]

In [438]:
display(training_data.tail(5))
display(testing_data.head(5))

,S&P_Close,S&P_Volume,NASDAQ_Close,NASDAQ_Volume,DJIA_Close,DJIA_Volume,SPY_div,QQQ_div,DIA_div,ten_yr,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-13,387.799866,56551000.0,331.755798,30374000.0,310.814270,3391700.0,0.017971,0.005254,0.019903,1.623,...,False,True,False,False,False,False,False,False,False,False
2021-04-14,386.475525,61659900.0,327.772034,42564300.0,311.497070,3966800.0,0.018032,0.005318,0.019859,1.638,...,False,True,False,False,False,False,False,False,False,False
2021-04-15,390.627136,60229800.0,332.739532,36342800.0,314.182678,3881800.0,0.017841,0.005238,0.019689,1.530,...,False,True,False,False,False,False,False,False,False,False
2021-04-16,391.932800,82037300.0,333.129181,40860100.0,315.596985,3748900.0,0.017781,0.005232,0.018048,1.573,...,False,True,False,False,False,False,False,False,False,False
2021-04-19,390.007202,78498500.0,330.080505,34533100.0,314.572205,3222800.0,0.017869,0.005281,0.018107,1.601,...,False,True,False,False,False,False,False,False,False,False


,S&P_Close,S&P_Volume,NASDAQ_Close,NASDAQ_Volume,DJIA_Close,DJIA_Volume,SPY_div,QQQ_div,DIA_div,ten_yr,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-20,387.151703,81851800.0,327.674652,38081800.0,312.144318,3762900.0,0.018001,0.005319,0.018248,1.562,...,False,True,False,False,False,False,False,False,False,False
2021-04-21,390.814972,66793000.0,330.479828,36598100.0,315.080048,4191200.0,0.017832,0.005274,0.018078,1.564,...,False,True,False,False,False,False,False,False,False,False
2021-04-22,387.245605,97582800.0,326.496094,45936100.0,312.144318,4962500.0,0.017996,0.005339,0.018248,1.554,...,False,True,False,False,False,False,False,False,False,False
2021-04-23,391.444336,73209200.0,330.606537,36381900.0,314.276764,3613900.0,0.017803,0.005272,0.018124,1.567,...,False,True,False,False,False,False,False,False,False,False
2021-04-26,392.261475,52182400.0,332.759094,30812000.0,313.759735,2090900.0,0.017766,0.005238,0.018154,1.570,...,False,True,False,False,False,False,False,False,False,False


### Normalize inputs

In [439]:
from sklearn.preprocessing import StandardScaler

In [487]:
# Make sure we only fit on training_data and explanatory variables
targets = ['S&P_next_ret', 'NASDAQ_next_ret', 'DJIA_next_ret']
dummies = [f'month_{month}' for month in range(1,13)]
columns_to_ignore = [] #['NASDAQ_next_ret', 'DJIA_next_ret']
columns_to_ignore.extend(dummies)
features = [column for column in training_data.columns if column not in targets and column not in columns_to_ignore]

scaler = StandardScaler()
scaler.fit(training_data[features]) # Fitting on training data

train_scaled = training_data.copy()
test_scaled = testing_data.copy()

train_scaled[features] = scaler.transform(training_data[features])
test_scaled[features] = scaler.transform(testing_data[features])

# Save info on standardization for later
variables_mu = pd.Series(scaler.mean_, index=features)
variables_sd = pd.Series(scaler.scale_, index=features)

### Linear Regression

In [488]:
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, root_mean_squared_error

We are going to test with and without ridge (which will help reduce the impact of collinearity)

In [489]:
# Function to get print results from the models
def eval_and_report(y_true, y_pred, model_name):
    print(f"{model_name:18s} | R^2: {r2_score(y_true, y_pred):.4f} | RMSE: {root_mean_squared_error(y_true, y_pred):.6f}")

In [490]:
# Models
results = {}

for target in targets:
    print(f"\n=== Target: {target} ===")
    X_train = train_scaled[features].copy()
    y_train = train_scaled[target].copy()
    X_test = test_scaled[features].copy()
    y_test = test_scaled[target].copy()

    # 1. Ordinary Least Squares (OLS)
    ols = LinearRegression()
    ols.fit(X_train, y_train)
    yhat_ols = ols.predict(X_test)
    eval_and_report(y_test, yhat_ols, "OLS")

    # Print top coefficients
    ols_coef = pd.Series(ols.coef_, index=features).sort_values(key=np.abs, ascending=False)
    print("Top OLS coeffs:\n", ols_coef.head(10))

    # 2. Ridge with CV over alphas (time-series CV)
    tscv = TimeSeriesSplit(n_splits=5)
    alphas = np.logspace(-4, 3, 30)

    ridge = RidgeCV(alphas=alphas, cv=tscv, fit_intercept=True)
    ridge.fit(X_train, y_train)
    yhat_ridge = ridge.predict(X_test)
    eval_and_report(y_test, yhat_ridge, f"Ridge (alpha={ridge.alpha_:.4g})")

    # Print top coefficients
    ridge_coef = pd.Series(ridge.coef_, index=features).sort_values(key=np.abs, ascending=False)
    print("Top Ridge coeffs:\n", ridge_coef.head(10))

    # Store for later use
    results[target] = {
        "ols_model": ols,
        "ridge_model": ridge,
        "ols_coefs": ols_coef,
        "ridge_coefs": ridge_coef,
        "yhat_ols": pd.Series(yhat_ols, index=y_test.index, name=f"{target}_OLS_pred"),
        "yhat_ridge": pd.Series(yhat_ridge, index=y_test.index, name=f"{target}_Ridge_pred"),
    }


=== Target: S&P_next_ret ===
OLS                | R^2: -8.4360 | RMSE: 0.136893
Top OLS coeffs:
 DJIA_Close              -0.360966
S&P_Close                0.249343
S&P_log_price           -0.232800
DJIA_log_price           0.231632
DJIA_3m_rolling_price    0.226790
S&P_3m_rolling_price    -0.226047
S&P_trend_dist_price    -0.102531
GDP                      0.096522
DJIA_trend_dist_price    0.081598
S&P_trend_speed_price    0.046233
dtype: float64
Ridge (alpha=1000) | R^2: 0.0050 | RMSE: 0.044452
Top Ridge coeffs:
 DJIA_Volume                -0.007322
ten_yr                     -0.006341
DIA_div                     0.005024
NASDAQ_Volume              -0.003867
SPY_div                     0.003299
NASDAQ_3m_rolling_volume    0.003098
NASDAQ_Close                0.002868
DJIA_mom_3m                -0.002833
DJIA_trend_dist_price      -0.002751
BAMLC0A4CBBB               -0.002720
dtype: float64

=== Target: NASDAQ_next_ret ===
OLS                | R^2: -2.4882 | RMSE: 0.112984
Top OLS c

Let's try OLS again with PCA

In [491]:
from sklearn.decomposition import PCA

In [492]:
pca = PCA(n_components=.95).fit(train_scaled) # keep 95% of variance and fit to training set
train_pca = pca.transform(train_scaled)
test_pca = pca.transform(test_scaled)

In [493]:
# PCA
for target in targets:
    print(f"\n=== Target: {target} ===")
    y_train = train_scaled[target].copy()
    y_test  = test_scaled[target].copy()

    # 1) OLS on PCA components
    ols_pca = LinearRegression()
    ols_pca.fit(train_pca, y_train)
    yhat_pca = ols_pca.predict(test_pca)
    eval_and_report(y_test, yhat_pca, "OLS+PCA")

    results[target].update({
        "pca_model": ols_pca,
        "yhat_pca": pd.Series(yhat_pca, index=y_test.index, name=f"{target}_PCA_pred")
    })


=== Target: S&P_next_ret ===
OLS+PCA            | R^2: 0.0499 | RMSE: 0.043437

=== Target: NASDAQ_next_ret ===
OLS+PCA            | R^2: 0.0444 | RMSE: 0.059137

=== Target: DJIA_next_ret ===
OLS+PCA            | R^2: 0.0824 | RMSE: 0.037906


Plot these results

In [494]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [495]:
# Pairs: (pretty name, next-return target key)
pairs = [
    ("S&P 500", "S&P_next_ret"),
    ("NASDAQ",  "NASDAQ_next_ret"),
    ("DJIA",    "DJIA_next_ret"),
]

def get_pred(results_dict, target_key, kind):
    """
    kind: 'ols', 'ridge', or 'pca'
    Prefer unstandardized preds if present, otherwise fall back to raw.
    """
    unstd_key = f"yhat_{kind}_unstd"
    std_key   = f"yhat_{kind}"
    if target_key in results_dict:
        if unstd_key in results_dict[target_key]:
            return results_dict[target_key][unstd_key]
        elif std_key in results_dict[target_key]:
            return results_dict[target_key][std_key]
    return None

fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.06,
    subplot_titles=[p[0] for p in pairs]
)

for i, (label, next_col) in enumerate(pairs, start=1):
    if next_col not in testing_data.columns:
        continue

    # True = next-period return from testing_data (unscaled)
    y_true = testing_data[next_col].dropna().sort_index().rename("True")

    # Predictions for the same target
    yhat_ols   = get_pred(results, next_col, "ols")
    yhat_ridge = get_pred(results, next_col, "ridge")
    yhat_pca   = get_pred(results, next_col, "pca")   # <-- NEW

    parts = [y_true]
    if yhat_ols is not None:
        parts.append(yhat_ols.rename("OLS").reindex(y_true.index))
    if yhat_ridge is not None:
        parts.append(yhat_ridge.rename("Ridge").reindex(y_true.index))
    if yhat_pca is not None:
        parts.append(yhat_pca.rename("PCA").reindex(y_true.index))  # <-- NEW

    df = pd.concat(parts, axis=1).dropna(how="any")
    if df.empty:
        continue

    show_leg = (i == 1)

    # True next_ret
    fig.add_trace(
        go.Scatter(x=df.index, y=df["True"], name="True next_ret",
                   mode="lines", line=dict(width=1.6),
                   showlegend=show_leg, legendgroup="true"),
        row=i, col=1
    )
    # OLS prediction
    if "OLS" in df:
        fig.add_trace(
            go.Scatter(x=df.index, y=df["OLS"], name="OLS prediction",
                       mode="lines", line=dict(width=1.4, dash="dash"),
                       showlegend=show_leg, legendgroup="ols"),
            row=i, col=1
        )
    # Ridge prediction
    if "Ridge" in df:
        fig.add_trace(
            go.Scatter(x=df.index, y=df["Ridge"], name="Ridge prediction",
                       mode="lines", line=dict(width=1.4, dash="dot"),
                       showlegend=show_leg, legendgroup="ridge"),
            row=i, col=1
        )
    # PCA prediction  <-- NEW
    if "PCA" in df:
        fig.add_trace(
            go.Scatter(x=df.index, y=df["PCA"], name="PCA prediction",
                       mode="lines", line=dict(width=1.4, dash="longdash"),
                       showlegend=show_leg, legendgroup="pca"),
            row=i, col=1
        )

fig.update_layout(
    title="Testing: True Next-Period Log Returns vs OLS, Ridge & PCA Predictions",
    height=900,
    hovermode="x unified",
    template="plotly_white",
    margin=dict(t=80, r=30, b=80, l=70),
    legend=dict(orientation="h", yanchor="top", y=-0.12, xanchor="left", x=0)
)

for r in range(1, 4):
    fig.update_yaxes(title_text="Log return", row=r, col=1)

# Range tools on the bottom axis
fig.update_xaxes(
    rangeselector=dict(buttons=[
        dict(step="all", label="All"),
        dict(count=3, step="year", stepmode="backward", label="3Y"),
        dict(count=1, step="year", stepmode="backward", label="1Y"),
        dict(count=6, step="month", stepmode="backward", label="6M"),
        dict(count=1, step="month", stepmode="backward", label="1M"),
    ]),
    rangeslider=dict(visible=True),
    row=3, col=1
)

fig.show()


# It looks like this may be an ok base for mu. Let's try to build sigma.

In [496]:
vol_data = s_p.join([nasdaq, djia])
vol_data

Price,S&P_Close,S&P_Volume,NASDAQ_Close,NASDAQ_Volume,DJIA_Close,DJIA_Volume
Date,,,,,,
1993-01-29,24.313030,1003200,NaN,NaN,NaN,NaN
1993-02-01,24.485970,480500,NaN,NaN,NaN,NaN
1993-02-02,24.537834,201300,NaN,NaN,NaN,NaN
1993-02-03,24.797218,529400,NaN,NaN,NaN,NaN
1993-02-04,24.900988,531500,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-09-15,659.082703,63772400,591.679993,44360300.0,458.759308,4862200.0
2025-09-16,658.175232,61169000,591.179993,36942100.0,457.472961,4526100.0
2025-09-17,657.357544,101952200,590.000000,69384800.0,459.945923,5998000.0


In [497]:
# First, make log prices / volumes of our data, then log normal assumptions are better and everything is additive
vol_data['S&P_log_price'] = np.log(vol_data['S&P_Close'])
vol_data['NASDAQ_log_price'] = np.log(vol_data['NASDAQ_Close'])
vol_data['DJIA_log_price'] = np.log(vol_data['DJIA_Close'])

vol_data['S&P_log_volume'] = np.log(data['S&P_Volume'])
vol_data['NASDAQ_log_volume'] = np.log(data['NASDAQ_Volume'])
vol_data['DJIA_log_volume'] = np.log(data['DJIA_Volume'])

# Log returns
vol_data['S&P_ret'] = vol_data['S&P_log_price'].diff(days)
vol_data['NASDAQ_ret'] = vol_data['NASDAQ_log_price'].diff(days)
vol_data['DJIA_ret'] = vol_data['DJIA_log_price'].diff(days)

# Next month log returns
vol_data[['S&P_next_ret','NASDAQ_next_ret','DJIA_next_ret']] = vol_data[['S&P_ret','NASDAQ_ret','DJIA_ret']].shift(-days).dropna()

# Residuals from previous
vol_data['S&P_next_ret'] = vol_data['S&P_next_ret'] - yhat_pca


# TODO: Change this, ripped straight from GPT as a quick test

In [498]:
EPS = 1e-12

# If you already have a rolling_mean helper, keep it. Otherwise:
def rolling_mean(s, w):
    return s.rolling(w, min_periods=max(2, int(w*0.6))).mean()

def ewma_vol(r, lam=0.94):
    # EWMA variance per RiskMetrics: sigma_t^2 = (1-lam)*r_{t-1}^2 + lam*sigma_{t-1}^2
    # Use pandas ewm for convenience
    return r.pow(2).ewm(alpha=(1-lam), adjust=False).mean().clip(lower=0)

def rolling_autocorr(x, lag=1, window=63):
    # Rolling autocorrelation of x at a given lag
    # For stability, require at least ~60% of window
    minp = max(10, int(window*0.6))
    x0 = x
    x1 = x.shift(lag)
    return x0.rolling(window, min_periods=minp).corr(x1)

def realized_quarticity(r, window=63):
    # 3-month robust quarticity proxy (if daily): sum r^4 * (n / 3) approximation
    # Here we simply provide rolling sum of r^4; scaling optional depending on use
    minp = max(10, int(window*0.6))
    return (r.pow(4)).rolling(window, min_periods=minp).sum()

def build_vol_features(data, prefix, day_w=21, qtr_w=63, yr_w=252, ewma_lambda=0.94):
    """
    Expects:
      data[f'{prefix}_log_price'] (daily log price)
      data[f'{prefix}_log_volume'] (daily log volume)
    Produces a suite of volatility-centric features for that prefix.
    """
    lp = data[f"{prefix}_log_price"]
    lv = data.get(f"{prefix}_log_volume", None)

    # Daily log return
    r = lp.diff()  # already log-price, so diff = log-return

    # --- Realized volatility proxies ---
    data[f"{prefix}_rv_1m"]  = r.rolling(day_w, min_periods=int(day_w*0.6)).var().clip(lower=0)          # variance
    data[f"{prefix}_rv_3m"]  = r.rolling(qtr_w, min_periods=int(qtr_w*0.6)).var().clip(lower=0)
    data[f"{prefix}_rv_1y"]  = r.rolling(yr_w,  min_periods=int(yr_w*0.6)).var().clip(lower=0)
    data[f"{prefix}_absrv_1m"] = r.abs().rolling(day_w, min_periods=int(day_w*0.6)).mean()               # mean |r|
    data[f"{prefix}_absrv_3m"] = r.abs().rolling(qtr_w, min_periods=int(qtr_w*0.6)).mean()

    # EWMA volatility (RiskMetrics-style)
    data[f"{prefix}_ewma_var"] = ewma_vol(r, lam=ewma_lambda)
    data[f"{prefix}_ewma_vol"] = np.sqrt(data[f"{prefix}_ewma_var"])

    # Volatility-of-volatility (how fast vol is changing)
    data[f"{prefix}_vol_speed_1w"] = data[f"{prefix}_rv_3m"].diff(5)                                      # weekly change in 3m var
    data[f"{prefix}_vol_mom_1m"]   = data[f"{prefix}_rv_3m"] - data[f"{prefix}_rv_1m"]                    # 3m vs 1m
    data[f"{prefix}_vol_mom_1y"]   = data[f"{prefix}_rv_1y"] - data[f"{prefix}_rv_3m"]

    # Volatility clustering proxies
    data[f"{prefix}_acf_sqret_lag1_3m"] = rolling_autocorr(r.pow(2), lag=1, window=qtr_w)
    data[f"{prefix}_acf_absret_lag1_3m"] = rolling_autocorr(r.abs(), lag=1, window=qtr_w)

    # Leverage effect proxy (contemporaneous corr between return and next day's vol)
    # Negative returns often precede higher vol; we proxy with corr(r_t, |r|_{t+1})
    data[f"{prefix}_lev_proxy_3m"] = r.rolling(qtr_w, min_periods=int(qtr_w*0.6)).corr(r.abs().shift(-1))

    # Quarticity (heavy tails proxy)
    data[f"{prefix}_quarticity_3m"] = realized_quarticity(r, window=qtr_w)

    # Ratio features (normalized vol levels)
    data[f"{prefix}_vol_ratio_1m_3m"] = (data[f"{prefix}_rv_1m"] / (data[f"{prefix}_rv_3m"] + EPS))
    data[f"{prefix}_vol_ratio_3m_1y"] = (data[f"{prefix}_rv_3m"] / (data[f"{prefix}_rv_1y"] + EPS))
    data[f"{prefix}_ewma_over_3m"]    = (data[f"{prefix}_ewma_var"] / (data[f"{prefix}_rv_3m"] + EPS))

    # Price–volatility relation: distance from trend as a stress proxy
    data[f"{prefix}_price_trend_3m"]  = rolling_mean(lp, qtr_w)
    data[f"{prefix}_price_trend_dist"] = lp - data[f"{prefix}_price_trend_3m"]
    # Volatility when far below trend often spikes; include interaction
    data[f"{prefix}_vol_x_trend_dist"] = data[f"{prefix}_rv_1m"] * data[f"{prefix}_price_trend_dist"]

    # Volume–volatility links (if volume available)
    if lv is not None:
        dv = lv.diff()  # log-volume change
        data[f"{prefix}_vlm_var_1m"] = dv.rolling(day_w, min_periods=int(day_w*0.6)).var().clip(lower=0)
        data[f"{prefix}_vlm_var_3m"] = dv.rolling(qtr_w, min_periods=int(qtr_w*0.6)).var().clip(lower=0)
        # Corr between |r| and volume changes (vol–volume clustering)
        data[f"{prefix}_corr_absr_dlv_3m"] = r.abs().rolling(qtr_w, min_periods=int(qtr_w*0.6)).corr(dv)
        # Volume surprise proxy: current vs 3m trend
        data[f"{prefix}_vlm_trend_3m"] = rolling_mean(lv, qtr_w)
        data[f"{prefix}_vlm_trend_dist"] = lv - data[f"{prefix}_vlm_trend_3m"]
        # Vol reacts to volume surprises
        data[f"{prefix}_vol_x_vlm_surprise"] = data[f"{prefix}_rv_1m"] * data[f"{prefix}_vlm_trend_dist"]

    # Optional: implied vs realized vol spread if you have VIX-like series
    # if f"{prefix}_impl_vol" in data.columns:
    #     data[f"{prefix}_ivr_spread"] = data[f"{prefix}_impl_vol"]**2 - data[f"{prefix}_rv_1m"]

    # Forward-looking realized vol target example (if needed)
    # data[f"{prefix}_fwd_rv_1m"] = r.shift(-1).rolling(day_w, min_periods=int(day_w*0.6)).var()

    return data

# ---- Apply to all indices ----
IDX_PREFIXES = ["S&P", "NASDAQ", "DJIA"]
for pfx in IDX_PREFIXES:
    vol_data = build_vol_features(vol_data, pfx, day_w=21, qtr_w=63, yr_w=252, ewma_lambda=0.94)

# ---- Cross-index spillover features (optional but useful) ----
# Differences/spreads in contemporaneous vol across indices capture contagion/regime moves
vol_data["SPX_minus_NDX_vol_1m"] = vol_data["S&P_rv_1m"] - vol_data["NASDAQ_rv_1m"]
vol_data["SPX_minus_DJIA_vol_1m"] = vol_data["S&P_rv_1m"] - vol_data["DJIA_rv_1m"]
vol_data["NDX_minus_DJIA_vol_1m"] = vol_data["NASDAQ_rv_1m"] - vol_data["DJIA_rv_1m"]

# A simple global vol factor: first PC of 3m realized vars (if you want a single factor)
try:
    _X = vol_data[["S&P_rv_3m", "NASDAQ_rv_3m", "DJIA_rv_3m"]].dropna()
    _Xc = (_X - _X.mean()) / (_X.std(ddof=0) + EPS)
    # first PC (no scikit-learn to keep it lightweight)
    U, S, Vt = np.linalg.svd(_Xc.values, full_matrices=False)
    gvol = pd.Series(U[:, 0]*S[0], index=_X.index, name="global_vol_pc1")
    vol_data["global_vol_pc1"] = gvol.reindex(data.index)
except Exception:
    pass

vol_data.dropna(inplace=True)

In [499]:
vol_data

Price,S&P_Close,S&P_Volume,NASDAQ_Close,NASDAQ_Volume,DJIA_Close,DJIA_Volume,S&P_log_price,NASDAQ_log_price,DJIA_log_price,S&P_log_volume,...,DJIA_vlm_var_1m,DJIA_vlm_var_3m,DJIA_corr_absr_dlv_3m,DJIA_vlm_trend_3m,DJIA_vlm_trend_dist,DJIA_vol_x_vlm_surprise,SPX_minus_NDX_vol_1m,SPX_minus_DJIA_vol_1m,NDX_minus_DJIA_vol_1m,global_vol_pc1
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-20,387.151703,81851800,327.674652,38081800.0,312.144318,3762900.0,5.958817,5.792021,5.743466,18.220421,...,0.053789,0.107425,0.118041,15.152043,-0.011342,-3.868883e-07,-0.000071,0.000012,0.000084,0.412036
2021-04-21,390.814972,66793000,330.479828,36598100.0,315.080048,4191200.0,5.968234,5.800546,5.752827,18.017109,...,0.047094,0.107201,0.122881,15.160903,0.087595,3.217142e-06,-0.000060,0.000011,0.000071,0.424124
2021-04-22,387.245605,97582800,326.496094,45936100.0,312.144318,4962500.0,5.959059,5.788418,5.743466,18.396212,...,0.044451,0.106752,0.115979,15.175893,0.241527,8.757578e-06,-0.000067,0.000012,0.000080,0.412417
2021-04-23,391.444336,73209200,330.606537,36381900.0,314.276764,3613900.0,5.969843,5.800929,5.750274,18.108832,...,0.047115,0.108461,0.114102,15.184895,-0.084597,-3.137968e-06,-0.000051,0.000011,0.000062,0.407980
2021-04-26,392.261475,52182400,332.759094,30812000.0,313.759735,2090900.0,5.971929,5.807419,5.748628,17.770256,...,0.060031,0.102982,0.179605,15.172565,-0.619460,-2.284884e-05,-0.000050,0.000011,0.000061,0.408586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-14,643.166870,59327500,579.890015,45425000.0,447.615112,4553200.0,6.466404,6.362838,6.103934,17.898584,...,0.088865,0.100060,0.420494,15.018578,0.312763,1.582230e-05,-0.000022,-0.000007,0.000015,0.803154
2025-08-15,641.661072,68592500,577.340027,49480200.0,448.249329,5584400.0,6.464060,6.358431,6.105350,18.043694,...,0.090003,0.099750,0.405569,15.024167,0.511321,2.501751e-05,-0.000022,-0.000006,0.000016,0.803666
2025-08-18,641.521423,43804900,577.109985,29831000.0,447.770691,3809100.0,6.463843,6.358033,6.104281,17.595256,...,0.097635,0.098377,0.438206,15.031300,0.121604,5.838557e-06,-0.000022,-0.000005,0.000017,0.806559


In [500]:
month_dummies = pd.get_dummies(vol_data.index.month, prefix="month")
month_dummies.set_index(vol_data.index, inplace=True)
vol_data = vol_data.join(month_dummies)

In [501]:

cutoff = math.floor(len(data)*.8)
training_data = data.iloc[:cutoff]
testing_data = data.iloc[cutoff:]
# Make sure we only fit on training_data and explanatory variables
targets = ['S&P_next_ret', 'NASDAQ_next_ret', 'DJIA_next_ret']
dummies = [f'month_{month}' for month in range(1,13)]
features = [column for column in training_data.columns if column not in targets and column not in dummies]

scaler = StandardScaler()
scaler.fit(training_data[features]) # Fitting on training data

train_scaled = training_data.copy()
test_scaled = testing_data.copy()

train_scaled[features] = scaler.transform(training_data[features])
test_scaled[features] = scaler.transform(testing_data[features])

# Save info on standardization for later
variables_mu = pd.Series(scaler.mean_, index=features)
variables_sd = pd.Series(scaler.scale_, index=features)

In [502]:
# Models
results = {}

for target in targets:
    print(f"\n=== Target: {target} ===")
    X_train = train_scaled[features].copy()
    y_train = train_scaled[target].copy()
    X_test = test_scaled[features].copy()
    y_test = test_scaled[target].copy()

    # 1. Ordinary Least Squares (OLS)
    ols = LinearRegression()
    ols.fit(X_train, y_train)
    yhat_ols = ols.predict(X_test)
    eval_and_report(y_test, yhat_ols, "OLS")

    # Print top coefficients
    ols_coef = pd.Series(ols.coef_, index=features).sort_values(key=np.abs, ascending=False)
    print("Top OLS coeffs:\n", ols_coef.head(10))

    # 2. Ridge with CV over alphas (time-series CV)
    tscv = TimeSeriesSplit(n_splits=5)
    alphas = np.logspace(-4, 3, 30)

    ridge = RidgeCV(alphas=alphas, cv=tscv, fit_intercept=True)
    ridge.fit(X_train, y_train)
    yhat_ridge = ridge.predict(X_test)
    eval_and_report(y_test, yhat_ridge, f"Ridge (alpha={ridge.alpha_:.4g})")

    # Print top coefficients
    ridge_coef = pd.Series(ridge.coef_, index=features).sort_values(key=np.abs, ascending=False)
    print("Top Ridge coeffs:\n", ridge_coef.head(10))

    # Store for later use
    results[target] = {
        "ols_model": ols,
        "ridge_model": ridge,
        "ols_coefs": ols_coef,
        "ridge_coefs": ridge_coef,
        "yhat_ols": pd.Series(yhat_ols, index=y_test.index, name=f"{target}_OLS_pred"),
        "yhat_ridge": pd.Series(yhat_ridge, index=y_test.index, name=f"{target}_Ridge_pred"),
    }


=== Target: S&P_next_ret ===


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values